# Lattice based cryptography

In [1]:
import numpy as np
from SVP_utils import SVPoint, read_basis, gram_schmidt, generate_basis_vectors, generate_random_points, augment, find_difference, find_modified_average, find_random, write_output
import random

In [2]:
B = np.array([
 [37, 20, 96, 20, 34, 64, 82, 56, 47, 21, 50, 49],
 [39, 24, 19, 49, 82, 97, 88, 84, 41, 51, 36, 74],
 [19, 56, 37, 73,  4, 12, 72, 18, 46,  8, 54, 94],
 [13, 46, 26,  8, 83, 71, 45, 84, 21, 32, 53, 80],
 [65, 39, 25, 56, 52, 44, 84, 30, 69, 33, 13,  5],
 [59, 56, 90,  1, 42, 58, 90, 92,  2,  6,  7, 80],
 [18, 14, 26, 31, 91, 93, 77, 64, 95, 36, 23,  5],
 [11, 58, 22, 51, 90, 13, 93, 43, 21, 81, 12, 77],
 [42, 65, 99,  6, 23, 43, 94, 30, 37, 66, 34, 66],
 [99, 31, 24, 44, 18, 58, 17, 27, 70, 88, 59, 11],
 [30, 43, 21, 70, 48, 47, 13, 93, 94, 48, 69, 58],
 [ 7, 12, 94, 88, 59, 95, 43, 62, 71, 36, 91, 70]
])

In [3]:
B = read_basis('../task/latticeBasis.txt')

In [4]:
s = 0
for b in B:
    print(np.linalg.norm(b))
    s += np.linalg.norm(b)

print('average:', s/12)

184.52100151473275
216.43936795324458
172.61228229763952
186.30619957478604
167.77067681809
207.12073773526396
199.96749735894582
195.9897956527329
197.9318064384802
185.00270268296083
201.70770932217738
233.43093196918014
average: 195.73339244318618


In [5]:
# Upper bound on the length of the shortest vector by minkowski's theorem
n = len(B[0])
print(f"Upper bound by Minkowski's theorem: {np.sqrt(n) * (np.linalg.det(B) ** (1/n))}")


# Estimation of the length of the shortest vector by https://www.latticechallenge.org/svp-challenge/#
from math import factorial

g_term = factorial( (n/2+1) -1 ) ** (1/n)
print(f"Estimation from latticechallenge.org: {1.05 * g_term / np.sqrt(np.pi) * (np.linalg.det(B) ** (1/n))}" )

Upper bound by Minkowski's theorem: 230.47420094416253
Estimation from latticechallenge.org: 68.1957168688578


In [6]:
def sieve(B):
    """
    Function to sieve for short vectors on a lattice with basis B
    
    Parameters:
    B (np.array): Input lattice basis vectors
    
    Returns:
    np.array: short vector on the lattice
    int: length of the short vector
    """
    # 0 Orthogonalise the basis with gram-schmidt
    # B_gs = gram_schmidt(B)

    # 1 Generate n points on the lattice
    n = 150

    # points = generate_basis_vectors(B, n)
    points = generate_random_points(B, n, l=-4, h=5)

    # 2 Replace the points with new ones until done
    # for i in range(80): # To find the shortest vector, unlikely to terminate
    while points[0].norm > 72.1: # To find the vector of length 72.08. If it doesn't terminate within 15s, rerun.
        
        print(f"average norm in points: {int(sum([p.norm for p in points])/len(points))}, shortest = {points[0].norm}")
        # print(points[0])
        assert (points[0].p == np.dot(B, points[0].x)).all()

        f = random.choice([find_difference, find_modified_average])
        points = augment(points, B, n, 0.85, f, timeout=10)
        # points = augment(points, B, n, 0.8, find_modified_average)
        # points = augment(points, B, n, 0.8, find_difference)

    return points[0]

SV = sieve(B)
assert (SV.p == np.dot(B, SV.x)).all()
print(f"The shortest vector:\n{SV}")

average norm in points: 1571, shortest = 417.7439407100958
average norm in points: 1252, shortest = 417.7439407100958
average norm in points: 1076, shortest = 395.8219296602956
average norm in points: 973, shortest = 395.8219296602956
average norm in points: 870, shortest = 334.5325096309774
average norm in points: 780, shortest = 259.32412151591296
average norm in points: 707, shortest = 259.32412151591296
average norm in points: 661, shortest = 259.32412151591296
average norm in points: 596, shortest = 196.7663589133061
average norm in points: 560, shortest = 196.7663589133061
average norm in points: 507, shortest = 196.7663589133061
average norm in points: 473, shortest = 196.7663589133061
average norm in points: 430, shortest = 185.24848177515517
average norm in points: 394, shortest = 185.24848177515517
average norm in points: 367, shortest = 185.24848177515517
average norm in points: 347, shortest = 185.24848177515517
average norm in points: 321, shortest = 164.59040069214242
ave

In [7]:
write_output(B, SV.p, SV.norm, SV.x, 'SVP output.txt')